In [16]:
import torch
import torch.nn as nn

In [17]:
class VGG( nn.Module ):
    def __init__( self, features, num_classes, init_weights ):
        super( VGG, self ).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout( p = 0.5 ),
            nn.Linear( 512 * 7 * 7, 2048 ),
            nn.ReLU( inplace=True ),
            nn.Dropout( p = 0.5 ),
            nn.Linear( 2048, 2048 ),
            nn.ReLU( inplace=True ),
            nn.Linear( 2048, num_classes ),
        )
        if init_weights:
            self._initialize_weights()
            
    def forward( self, x ):
        x = self.features( x )                  # batch × 3 × 224 × 224 -> batch ×512 × 7 × 7
        x = torch.flatten( x, start_dim = 1 )   # batch × 512*7*7
        return self.classifier( x )
        
        
        
    def _initialize_weights( self ):            # 初始化权重的方法
        for m in self.modules():
            if isinstance( m, nn.Conv2d ):      # 如果m是卷积层的话，初始化它的权重和偏值
                nn.init.kaiming_normal_( m.weight, mode='fan_out' )
                if m.bias is not None:
                    nn.init.constant_( m.bias, 0 )
            elif isinstance( m, nn.Linear ):
                nn.init.xavier_uniform( m.weight )
                nn.init.constant_( m.bias, 0 )
        

In [18]:
networks = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

def make_features( ntw: list ):
    layers = []
    in_channels = 3         # RGB -> 3
    for v in ntw:
        if v == 'M':
            layers += [ nn.MaxPool2d( kernel_size=2, stride=2 ) ]
        else:               # 是数字就是卷积 -> 改变长宽
            conv2d = nn.Conv2d( in_channels, v, kernel_size=3, padding = 1 )
            layers += [ conv2d, nn.ReLU( inplace=True ) ]
            in_channels = v                     # 下一次的inchannel直接变为这一次的输出深度
    
    return nn.Sequential( *layers )

实例化给定的模型

In [19]:
def vgg( model_name, num_classes, init_weights ):
    assert model_name in networks, "Not in dict!" 
    ntw = networks[ model_name ]

    model = VGG( make_features( ntw ), num_classes, init_weights )
    return model

In [20]:
vgg_model = vgg( model_name='vgg16', num_classes = 5, init_weights = True )

C:\Users\EC319\AppData\Local\Temp\ipykernel_14904\4093306880.py:31: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform( m.weight )
